In [1]:
! mkdir ~/.kaggle

In [2]:
! cp kaggle.json ~/.kaggle/

In [3]:
! chmod 600 ~/.kaggle/kaggle.json


In [4]:
! kaggle datasets download imano00/dataset3modified


 99% 112M/113M [00:07<00:00, 19.2MB/s]
100% 113M/113M [00:07<00:00, 16.4MB/s]


In [5]:
# Put on the same directory
from zipfile import ZipFile

# specifying the name of the zip file
file = "dataset3modified.zip"

# open the zip file in read mode
with ZipFile(file, 'r') as zip:
    # list all the contents of the zip file
    zip.printdir()

    # extract all files
    print('extraction...')
    zip.extractall()
    print('Done!')

Streaming output truncated to the last 5000 lines.
dataset(3) - modified/images/train/happy/Training_49789902.jpg 2022-12-17 10:41:58         1621
dataset(3) - modified/images/train/happy/Training_49830552.jpg 2022-12-17 10:41:58         1609
dataset(3) - modified/images/train/happy/Training_49832352.jpg 2022-12-17 10:41:58         1392
dataset(3) - modified/images/train/happy/Training_49844940.jpg 2022-12-17 10:41:58         1642
dataset(3) - modified/images/train/happy/Training_49848474.jpg 2022-12-17 10:41:58         1493
dataset(3) - modified/images/train/happy/Training_49883752.jpg 2022-12-17 10:41:58         1499
dataset(3) - modified/images/train/happy/Training_49890250.jpg 2022-12-17 10:41:58         1662
dataset(3) - modified/images/train/happy/Training_49897087.jpg 2022-12-17 10:41:58         1667
dataset(3) - modified/images/train/happy/Training_49897219.jpg 2022-12-17 10:41:58         1583
dataset(3) - modified/images/train/happy/Training_498981.jpg 2022-12-17 10:41:58     

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



Done!


In [17]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import os
import tensorflow as tf

from keras.preprocessing.image import load_img, img_to_array
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense,Input,Dropout,GlobalAveragePooling2D,Flatten,Conv2D,BatchNormalization,Activation,MaxPooling2D
from keras.models import Model,Sequential
from tensorflow.keras.optimizers import Adam,SGD,RMSprop
from sklearn.metrics import confusion_matrix,ConfusionMatrixDisplay
from tensorflow.keras.optimizers import Adam,SGD,RMSprop
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from sklearn.metrics import classification_report

In [18]:
train_set = ImageDataGenerator(rescale = 1./255).flow_from_directory('/content/dataset(3) - modified/images/train',
                                              target_size = (48,48),
                                              color_mode = "grayscale",
                                              batch_size=128,
                                              class_mode='categorical',
                                              shuffle=True)

test_set = ImageDataGenerator(rescale = 1./255).flow_from_directory('/content/dataset(3) - modified/images/validation',
                                              target_size = (48,48),
                                              color_mode = "grayscale",
                                              batch_size=128,
                                              class_mode='categorical',
                                              shuffle=False)


Found 58454 images belonging to 7 classes.
Found 7066 images belonging to 7 classes.


In [19]:
model = Sequential()

#1st CNN layer
model.add(Conv2D(64,(3,3),padding = 'same',input_shape = (48,48,1)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size = (2,2)))
model.add(Dropout(0.25))

#2nd CNN layer
model.add(Conv2D(128,(5,5),padding = 'same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size = (2,2)))
model.add(Dropout (0.25))

#3rd CNN layer
model.add(Conv2D(512,(3,3),padding = 'same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size = (2,2)))
model.add(Dropout (0.25))

#4th CNN layer
model.add(Conv2D(512,(3,3), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())

#Fully connected 1st layer
model.add(Dense(256))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.25))


# Fully connected layer 2nd layer
model.add(Dense(512))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.25))
# Fully connected layer 3rd layer
model.add(Dense(512))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.22))

model.add(Dense(7, activation='softmax'))



opt = Adam(learning_rate = 0.0001)
model.compile(optimizer=opt,loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_4 (Conv2D)           (None, 48, 48, 64)        640       
                                                                 
 batch_normalization_7 (Bat  (None, 48, 48, 64)        256       
 chNormalization)                                                
                                                                 
 activation_7 (Activation)   (None, 48, 48, 64)        0         
                                                                 
 max_pooling2d_4 (MaxPoolin  (None, 24, 24, 64)        0         
 g2D)                                                            
                                                                 
 dropout_7 (Dropout)         (None, 24, 24, 64)        0         
                                                                 
 conv2d_5 (Conv2D)           (None, 24, 24, 128)      

In [ ]:
reduce_learningrate = ReduceLROnPlateau(monitor='val_loss',
                              factor=0.2,
                              patience=3,
                              verbose=1,
                              min_delta=0.0001)

callbacks_list = [reduce_learningrate]

model.compile(loss='categorical_crossentropy',
              optimizer = Adam(learning_rate=0.001),
              metrics=['accuracy'])
history = model.fit(train_set,
                    steps_per_epoch=train_set.n//train_set.batch_size,
                    epochs=30,
                    validation_data = test_set,
                    validation_steps = test_set.n//test_set.batch_size,
                    callbacks=callbacks_list
                    )

Epoch 1/30
456/456 [==============================] - 39s 73ms/step - loss: 1.6199 - accuracy: 0.3747 - val_loss: 1.9351 - val_accuracy: 0.3290 - lr: 0.0010
Epoch 2/30
456/456 [==============================] - 43s 94ms/step - loss: 1.2450 - accuracy: 0.5228 - val_loss: 1.3239 - val_accuracy: 0.5121 - lr: 0.0010
Epoch 3/30
456/456 [==============================] - 33s 72ms/step - loss: 1.1223 - accuracy: 0.5742 - val_loss: 1.3376 - val_accuracy: 0.5223 - lr: 0.0010
Epoch 4/30
456/456 [==============================] - 33s 73ms/step - loss: 1.0340 - accuracy: 0.6106 - val_loss: 1.0539 - val_accuracy: 0.5959 - lr: 0.0010
Epoch 5/30
456/456 [==============================] - 34s 74ms/step - loss: 0.9630 - accuracy: 0.6354 - val_loss: 1.0751 - val_accuracy: 0.5963 - lr: 0.0010
Epoch 6/30
456/456 [==============================] - 36s 78ms/step - loss: 0.8977 - accuracy: 0.6651 - val_loss: 0.8981 - val_accuracy: 0.6753 - lr: 0.0010
Epoch 7/30
456/456 [==============================] - 33s 

In [ ]:
plt.figure(figsize=(10,5))
plt.subplot(1, 2, 1)
plt.suptitle('Optimizer : Adam', fontsize=10)
plt.ylabel('Loss', fontsize=12)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.legend(loc='upper right')

plt.subplot(1, 2, 2)
plt.ylabel('Accuracy', fontsize=12)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.legend(loc='lower right')
plt.show()

In [ ]:
import seaborn as sns
plt.figure (figsize = [8,8])
y_true = test_set.classes
y_pred =  model.predict(test_set)
y_pred= tf.argmax(y_pred, axis = 1).numpy()
cm = confusion_matrix(y_true = y_true, y_pred = y_pred)
sns.heatmap(cm, annot=True,cmap='Blues')
plt.title("matrix")
plt.show()


In [12]:
print(classification_report(y_true, y_pred))

              precision    recall  f1-score   support

           0       0.93      0.91      0.92       960
           1       0.96      0.89      0.93       111
           2       0.93      0.91      0.92      1018
           3       0.97      0.98      0.97      1825
           4       0.91      0.94      0.92      1216
           5       0.91      0.91      0.91      1139
           6       0.96      0.95      0.96       797

    accuracy                           0.94      7066
   macro avg       0.94      0.93      0.93      7066
weighted avg       0.94      0.94      0.94      7066



In [13]:
from keras.models import load_model

model.save('model.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [14]:
from keras.models import load_model

# Assuming 'model' is your Keras model
# Save the model to an HDF5 file
model.save('model.h5')